# Bayesian Regression & MLE for Student Performance Analysis

**Team member**: Hung Dinh - 19774520

**Theme**: Bayesian vs Frequentist Approach on Regression - Student Performance

**Github Repo**: https://github.com/hd54/stat447c

All commits are done by me and me only.

**Introduction**:

Performance have always been a concern for a lot of students regardless of education level, whether it be high school, university, or college. Good performance can mean greater opportunities for higher education, awards, and even jobs, so students want to be successful in their courses. However, there's always a disparity in students' performance, which can be seen in grade distributions of exams, homework, etc. It's possible that students' background or how they treat the class affect their performance. This project seeks to see how different factors contribute to students' performance (in particular, final exam score or course grade letter).

**Dataset**:

Student performance may vary throughout the years due to societal changes, introduction of new technologies (such as ChatGPT), etc. This leads me to choose some of the more recent datasets as possible candidates:

https://www.kaggle.com/datasets/lainguyn123/student-performance-factors

https://www.kaggle.com/datasets/joebeachcapital/students-performance

In [1]:
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
dataset_1 <- read.csv("StudentPerformanceFactors.csv")
head(dataset_1)

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>
1,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
2,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
3,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
4,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
5,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
6,19,88,Medium,Medium,Yes,8,89,Medium,Yes,3,Medium,Medium,Public,Positive,3,No,Postgraduate,Near,Male,71


In [3]:
dataset_2 <- read.csv("StudentsPerformance_with_headers.csv")
head(dataset_2)

,STUDENT.ID,Student.Age,Sex,Graduated.high.school.type,Scholarship.type,Additional.work,Regular.artistic.or.sports.activity,Do.you.have.a.partner,Total.salary.if.available,Transportation.to.the.university,...,Preparation.to.midterm.exams.1,Preparation.to.midterm.exams.2,Taking.notes.in.classes,Listening.in.classes,Discussion.improves.my.interest.and.success.in.the.course,Flip.classroom,Cumulative.grade.point.average.in.the.last.semester...4.00.,Expected.Cumulative.grade.point.average.in.the.graduation...4.00.,COURSE.ID,GRADE
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,STUDENT1,2,2,3,3,1,2,2,1,1,...,1,1,3,2,1,2,1,1,1,1
2,STUDENT2,2,2,3,3,1,2,2,1,1,...,1,1,3,2,3,2,2,3,1,1
3,STUDENT3,2,2,2,3,2,2,2,2,4,...,1,1,2,2,1,1,2,2,1,1
4,STUDENT4,1,1,1,3,1,2,1,2,1,...,1,2,3,2,2,1,3,2,1,1
5,STUDENT5,2,2,1,3,2,2,1,3,1,...,2,1,2,2,2,1,2,2,1,1
6,STUDENT6,2,2,2,3,2,2,2,2,1,...,1,1,1,2,1,2,4,4,1,2


**Approaches**:

Overall, the dataset contains a lot of discrete variables. For example, in the 2nd dataset, student age tends to be mostly 2, but this describes the range of 22-25 instead.

I can start simple with a comparative study of different types of regression using frequentist and Bayesian approach. I can do a comparative study on traditional (without penalty) and regularized regression for each approach. For frequentist approach, I can start by optimizing the number of variables used for regression through removing collinearity (i.e. we can use forward selection along with VIF analysis). I would expect to see regularized model to perform at least as well on predicting data due to the extra penalty added.

For Bayesian approach, we can use hierarchical model. It happens that we can treat the regularizer term as a prior, since it controls how much information we can learn from data, whereas frequentist approach will include an extra term as penalty value. 

If given enough time, I can also study interaction effects, but I can achieve a (hopefully) reasonable multilinear regression model.

**References**:
https://haines-lab.com/post/on-the-equivalency-between-the-lasso-ridge-regression-and-specific-bayesian-priors/